## DS/CMPSC 410 MiniProject #1

Learning Objectives
- Be able to transform columns of Big Data for feature engineering using DataFrame.
- Be able to perform simple data analytics using SQL Spark

In [ ]:
import pyspark
import csv

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, LongType
from pyspark.sql.functions import col, column
from pyspark.sql.functions import expr
from pyspark.sql.functions import split
from pyspark.sql import Row
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, IndexToString
from pyspark.ml.clustering import KMeans

In [ ]:
ss = SparkSession.builder.master("local").appName("DataFrameSQL").getOrCreate()

In [ ]:
Scanners_df = ss.read.csv("/storage/home/juy1/work/Darknet/scanners-dataset1-anon.csv", header= True, inferSchema=True )

## We can use printSchema() to display the schema of the DataFrame Scanners_df to see whether it was inferred correctly.

In [ ]:
Scanners_df.printSchema()

## Part A Transfosrm the feature "host_services_per_censys" into an array of services.
#### The original value of the column is a string that connects all the network services/ports open by a sourceIP/scanner.
#### The different services/ports open are connected by dash "-". For example, "81-161-2000" indicates the
#### scanner has three ports/services open: port 81, port 161, and port 2000.
#### Therefore, we want to use split to separate it into an array of ports/services open by each scanner.
#### This transformation is important because it enables features such as the following to be created:
#### (1) Whether the ports/services used by the scanner include those used by a known malware.
#### (2) Whether the ports/services used by a scanner overlaps with those used by another scanner.

## The original value of the column "host_services_per_censys" 

In [ ]:
Scanners_df.select("host_services_per_censys").show(10)

## We want to find the services/ports that most scanners have them open. There are multiple ways to find this information using Spark.  We will demonstrate one below, which involves the following steps:
1. Convert the DataFrame Scanners_df into an RDD
2. Map to each row of the DF-converted RDD to extract the column "host_services_per_censys". Save the mapping result in a new RDD (which contains only values of the column).
3. Use flatMap to split to the string (using "-" as the delimiter) to convert the RDD into an RDD of ports/services that are open on the host of the scanner.
4. Use map to generate a key-value pair RDD, where key is a port/service opens on a scanner, the value is 1.
5. Use reduceByKey to count the total number of scanners that have a specific port/service open.

In [ ]:
# Step 1: Convert the DataFrame Scanners_df into an RDD
Scanners_RDD = Scanners_df.rdd
Scanners_RDD.persist().take(5)

In [ ]:
# Step 2: Map to each row of the DF-converted RDD to extract the column "host_services_per_censys". Save the mapping result 
# in a new RDD (whih contains only values of the column)
Host_services_column = Scanners_RDD.map(lambda row: row.host_services_per_censys)
Host_services_column.persist().take(10)

## We can transform the string into a list of services (i.e., port numbers) that the scanner has opened using map.

In [ ]:
Host_services_rdd=Host_services_column.map(lambda string: string.split("-"))
Host_services_rdd.persist().take(10)

## Step 3: However, in order to count how many scanners are keeping a specific port/service open, it is easier to use flatMap (instead of map above) to "flatten" the results of splitting (using "-" as the delimiter to convert the input RDD into an RDD of all ports/services that are open on the host of each sourceIP.

In [ ]:
Host_services_f_list = Host_services_column.flatMap(lambda string: string.split("-"))

## Step 4: Use map to generate a key-value pair RDD, where key is a port/service opens on a scanner, the value is 1.

In [ ]:
Host_services_count = Host_services_f_list.map(lambda s: (s, 1) ) 
Host_services_count.persist().take(10)

## Step 5: Use reduceByKey to count the total number of scanners that have a specific port/service open.
## Exercise 1 (7 points): Complete the following two cells to do the following remaining steps
1. Calculate the total number of scanners that have each port/service open. 
2. Sort them in descending order of count so that we can see the port/services that are open for most scanners. Save the resulted file in a directory you specified.

In [ ]:
# Exercise 1-1
Host_services_total= Host_services_count.reduceByKey(lambda ???)
Host_services_total.persist().take(10)

In [ ]:
#Exercise 1-2
Count_Services = Host_services_total.map(lambda x: ??? ) )
Sorted_Count_Services = Count_Services.sortByKey(???)
Sorted_Count_Services.top(100)
Sorted_Count_Services.saveAsTextFile("????")

## Part B: We want to add a column to the original DataFrame such that the new column stores the array of services/ports opened by each scanner.
## From Spark programming viewpoint, this involves adding a column of a complex data type (e.g., Array of strings).
### This can be done by "selecting" from a DataFrame the result of applying split to a string using a delimiter (e.g., "-" in our case)
### Notice 1: This split applies to each row of the DataFrame.  Therefore, it needs to use "col" of Spark SQL to refer to a specific column of the row.
### Notice 2: The result of this is a new DataFrame with only one column, which is the result of select(split...)

In [ ]:
Scanners_df.select(split(col("host_services_per_censys"), "-")).show(10)

## Often, we are especially interested in scanners who have a specific port open because it may indicate that the scanner is a specific type of malware.  An example of this is port 23 used by the Mirai botnet.
## Whether a scanner has a specific port (e.g., port 23) open can be determined based on whether the array generated above by splitting "host_services_per_censys" contains the port (e.g., port 23).

In [ ]:
from pyspark.sql.functions import array_contains
Scanners_df.select(array_contains(split(col("host_services_per_censys"), "-"), "23")).show(40)

## Because the array of services is easier to process (than the original string), it is desirable to save it as an extra column in the original DataFrame (so that we do not need to spend time doing the "split" in the future.

## We can use DataFrame.withColumn to add a column to a Data Frame.  The code below attempts to add the two new columns we generated above:
- A column that contains the array of all hosts/ports opened by a scanner.
- A column that contains a Boolean value indicating whether the array of services contain a specific port (e.g., port 23).

### Exercise 2 (8 points) Did the list of columns include the previous column we added (i.e., "Services_Array")?
1. Check whether the last DF contains the two new columns?  Explain the result.
2. Fix the problem by typing your modified code in the code cell "Exercise 2 Solutions:". You should evaluate the cells below (makred as #Exercise 2 code). However, DO NOT MODIFY THE code in these cells. 
3. Explain why the solution works in the MarkDown Cell of "Answer to Exercise 2".

In [ ]:
# Exercise 2 (DO NOT CHANGE)
Scanners_df.withColumn("Services_Array", split(col("host_services_per_censys"), "-"))

In [ ]:
# Exercise 2 (DO NOT CHANGE)
Scanners_df.withColumn("Services_Array", split(col("host_services_per_censys"), "-")).show(1)

In [ ]:
# Exercise 2 (do not change)
Scanners_df.withColumn("Service_P23", array_contains(split(col("host_services_per_censys"), "-"), "23"))

In [ ]:
# Exercise 2 (do not change)
Scanners_df.withColumn("Service_P23", array_contains(split(col("host_services_per_censys"), "-"), "23")).show(1)

In [ ]:
# Exercise 2 (do not change)
Scanners_df.columns

In [ ]:
# Exercise 2  Solutions:


## Answer to Exercise 2:

## Part C. Create a Table so that we can post an SQL Query to the Table for simple Data Analytics
## Exercise 3 (6 points) Below are some examples of data analytics questions.  Which of them can be answered by an SQL query? 
1. Which country has the largest number of scanners who scan at least 10 ports?
2. Which service/port is open for the largest number of scanners (based on Censys)?
3. How many scanners includes port 21 in the list of ports it scanned?

## Answer to Exercise 3:
1. Which country has the largest number of scanners who scan at least 10 ports?
2. Which service/port is open for the largest number of scanners (based on Censys)?
3. How many scanners includes port 21 in the list of ports it scanned?

In [ ]:
Scanners_df3.createOrReplaceTempView("Scanners_sql_view")

## We can then submit an SQL Query to a View (i.e., temporary Table)

In [ ]:
sqlQuery1 = ss.sql("SELECT * FROM Scanners_sql_view where country = 'Thailand'")

In [ ]:
sqlQuery1.show(5)

In [ ]:
sqlQuery2 = ss.sql("SELECT * FROM Scanners_sql_view where num_ports_scanned > 20")
sqlQuery2.show(5)

In [ ]:
sqlQuery3 = ss.sql("SELECT count(sourceIP) FROM Scanners_sql_view where num_ports_scanned > 10")
sqlQuery3.show()

In [ ]:
sqlQuery4 = ss.sql("SELECT country, count(sourceIP) FROM Scanners_sql_view GROUP BY country ")
sqlQuery4.show(10)

In [ ]:
sqlQuery5 = ss.sql("SELECT country, count(sourceIP) FROM Scanners_sql_view WHERE Service_P23=True GROUP BY country")
sqlQuery5.show(5)

## SQL can also sort the query result by a specific column using ORDER BY.
## If we want to sort them by descreasing order, we use ORDER BY ... DESC.  (The default is ascending order.)

In [ ]:
sqlQuery6=ss.sql("SELECT country, count(sourceIP) FROM Scanners_sql_view WHERE Service_P23=True GROUP BY country \
ORDER BY count(sourceIP) DESC")
sqlQuery6.show(20)

### Exercise 4 (9 points) Write the SQL query to find the total number of scanners (by country) who scanned at least 5 ports, sort them in descending order of the count. Show the top 20 countries.

In [ ]:
sqlQuery4 = ss.sql("SELECT ??? FROM ??? WHERE ??? \
GROUP BY ??? ORDER BY ???  ???")
sqlQuery4.show(20)